In [2]:
from datetime import datetime
from datetime import timedelta
import shapefile
import xml.etree.ElementTree as ET
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import matplotlib.pyplot as plt
from pyproj import Proj, transform
from pymongo import MongoClient
%matplotlib inline

In [3]:
tree = ET.parse('/media/amassaro/external_2T/telecom_bigdata_2015/data/external_data/xml_incidenti_mar-apr_2015.xml')
root=tree.getroot()
child=list(root)[1]

In [4]:
acc_times=[]
place=[]
for j in child:
    if 'Latitudine' in j.attrib.keys() and 'Longitudine' in j.attrib.keys():
        place.append([float(j.attrib['Longitudine']),float(j.attrib['Latitudine'])])
        acc_times.append(j.attrib['DataOraIncidente'])

acc_times=np.array([datetime.strptime(t[:10]+' '+t[11:],'%Y-%m-%d %H:%M:%S') for t in acc_times])
acc_places=np.array(place)

In [8]:
shp='/media/amassaro/external_2T/telecom_bigdata_2015/data/tim_grid/roma-grid/intersection_Roma_W_GRIDIT_NEW'
crt=shapefile.Reader(shp)
geomet=crt.shapeRecords()


In [9]:
inProj = Proj(init='epsg:32632')
outProj = Proj(init='epsg:4326')

grid=np.array([[transform(inProj,outProj,c[0],c[1]) for c in g.shape.points] for g in geomet])
A=np.array([Polygon(g.shape.points).area for g in geomet])
A=A/min(A)
ids=np.array([g.record[0] for g in geomet])

In [15]:
acc_cells=np.zeros(len(acc_places))
I=np.zeros(len(grid))

for k in range(len(acc_places)):
    point=Point(acc_places[k])
    for i in range(len(grid)):
        if Polygon(grid[i]).contains(point):
            acc_cells[k]=i
            I[i]+=1
            break
I_n=I/A

In [26]:
out=open('percentiles_on_call_in_','w')
out.close()
out=open('percentiles_on_call_in_hourly_','w')
out.close()
CC=[]
db=MongoClient()
db=db['call_in_roma']
for grid_idx in np.where(I>0)[0]:    
    cur=db['callIn'].find({'square_id':ids[grid_idx]})
    time=[]
    calls_in=[]
    time_in=[]
    for c in cur:
        time.append(datetime.fromtimestamp(c['time_interval']))
        calls_in.append(c['n_of_calls'])
        time_in.append(c['call_time'])
    calls_in=np.array(calls_in)
    time=np.array(time)
    Idx=np.argsort(time)
    time=time[Idx]
    calls_in=calls_in[Idx]
    C=[]
    for t in acc_times[np.where(acc_cells==grid_idx)]:
        m=np.sum(time<t)
        if time[m]==t or m==0:
            C.append(calls_in[m])
        else:
            C.append(.5*(calls_in[m-1]+calls_in[m]))
    L_c=float(len(calls_in))
    ttime=np.array([t.replace(minute=0, second=0) for t in time])
    Time=np.sort(list(set(ttime)))
    Calls_in=np.array([np.sum(calls_in[np.where(ttime==t)]) for t in Time])
    
    
    CC=[]
    for t in acc_times[np.where(acc_cells==grid_idx)]:
        t=t.replace(minute=0, second=0)
        m=np.sum(Time<t)
        if Time[m]==t or m==0:
            CC.append(Calls_in[m])
        else:
            CC.append(.5*(Calls_in[m-1]+Calls_in[m]))
    L_cc=float(len(Calls_in))
    
    out=open('percentiles_on_call_in_','a')
    out.write(str(grid_idx)+',')
    
    
    for c in C:
        out.write(str(np.sum(calls_in<=c)/L_c)+',')
    out.write('\n')
    out.close()
    
    out=open('percentiles_on_call_in_hourly_','a')
    out.write(str(grid_idx)+',')
    for c in CC:
        out.write(str(np.sum(Calls_in<=c)/L_cc)+',')
    out.write('\n')
    out.close()
    
    

In [8]:
out=open('percentiles_on_call_out','w')
out.close()
out=open('percentiles_on_call_out_hourly','w')
out.close()
CC=[]
db=MongoClient()
db=db['call_out_roma']
for grid_idx in np.where(I>0)[0]:    
    cur=db['callOut'].find({'square_id':ids[grid_idx]})
    time=[]
    calls_in=[]
    time_in=[]
    for c in cur:
        time.append(datetime.fromtimestamp(c['time_interval']))
        calls_in.append(c['n_of_calls'])
        time_in.append(c['call_time'])
    calls_in=np.array(calls_in)
    time=np.array(time)
    Idx=np.argsort(time)
    time=time[Idx]
    calls_in=calls_in[Idx]
    C=[calls_in[np.sum(time<t)] for t in acc_times[np.where(acc_cells==grid_idx)]]
    L_c=float(len(calls_in))
    ttime=np.array([t.replace(minute=0, second=0) for t in time])
    Time=np.sort(list(set(ttime)))
    Calls_in=np.array([np.sum(calls_in[np.where(ttime==t)]) for t in Time])
    
    CC=[Calls_in[np.sum(Time<t.replace(minute=0, second=0))] for t in acc_times[np.where(acc_cells==grid_idx)]]
    L_cc=float(len(Calls_in))
    
    out=open('percentiles_on_call_out','a')
    out.write(str(grid_idx)+',')
    
    
    for c in C:
        out.write(str(np.sum(calls_in<=c)/L_c)+',')
    out.write('\n')
    out.close()
    
    out=open('percentiles_on_call_out_hourly','a')
    out.write(str(grid_idx)+',')
    for c in CC:
        out.write(str(np.sum(Calls_in<=c)/L_cc)+',')
    out.write('\n')
    out.close()


In [9]:
out=open('percentiles_on_sms_in','w')
out.close()
out=open('percentiles_on_sms_in_hourly','w')
out.close()
CC=[]
db=MongoClient()
db=db['sms_in_roma']
for grid_idx in np.where(I>0)[0]:    
    cur=db['smsIn'].find({'square_id':ids[grid_idx]})
    time=[]
    calls_in=[]
    time_in=[]
    for c in cur:
        time.append(datetime.fromtimestamp(c['time_interval']))
        calls_in.append(c['n_of_sms'])
    calls_in=np.array(calls_in)
    time=np.array(time)
    Idx=np.argsort(time)
    time=time[Idx]
    calls_in=calls_in[Idx]
    C=[calls_in[np.sum(time<t)] for t in acc_times[np.where(acc_cells==grid_idx)]]
    L_c=float(len(calls_in))
    ttime=np.array([t.replace(minute=0, second=0) for t in time])
    Time=np.sort(list(set(ttime)))
    Calls_in=np.array([np.sum(calls_in[np.where(ttime==t)]) for t in Time])
    
    CC=[Calls_in[np.sum(Time<t.replace(minute=0, second=0))] for t in acc_times[np.where(acc_cells==grid_idx)]]
    L_cc=float(len(Calls_in))
    
    out=open('percentiles_on_sms_in','a')
    out.write(str(grid_idx)+',')
    
    
    for c in C:
        out.write(str(np.sum(calls_in<=c)/L_c)+',')
    out.write('\n')
    out.close()
    
    out=open('percentiles_on_sms_in_hourly','a')
    out.write(str(grid_idx)+',')
    for c in CC:
        out.write(str(np.sum(Calls_in<=c)/L_cc)+',')
    out.write('\n')
    out.close()


In [10]:
out=open('percentiles_on_sms_out','w')
out.close()
out=open('percentiles_on_sms_out_hourly','w')
out.close()
CC=[]
db=MongoClient()
db=db['sms_out_roma']
for grid_idx in np.where(I>0)[0]:    
    cur=db['smsOut'].find({'square_id':ids[grid_idx]})
    time=[]
    calls_in=[]
    time_in=[]
    for c in cur:
        time.append(datetime.fromtimestamp(c['time_interval']))
        calls_in.append(c['n_of_sms'])
    calls_in=np.array(calls_in)
    time=np.array(time)
    Idx=np.argsort(time)
    time=time[Idx]
    calls_in=calls_in[Idx]
    C=[calls_in[np.sum(time<t)] for t in acc_times[np.where(acc_cells==grid_idx)]]
    L_c=float(len(calls_in))
    ttime=np.array([t.replace(minute=0, second=0) for t in time])
    Time=np.sort(list(set(ttime)))
    Calls_in=np.array([np.sum(calls_in[np.where(ttime==t)]) for t in Time])
    
    CC=[Calls_in[np.sum(Time<t.replace(minute=0, second=0))] for t in acc_times[np.where(acc_cells==grid_idx)]]
    L_cc=float(len(Calls_in))
    
    out=open('percentiles_on_sms_out','a')
    out.write(str(grid_idx)+',')
    
    
    for c in C:
        out.write(str(np.sum(calls_in<=c)/L_c)+',')
    out.write('\n')
    out.close()
    
    out=open('percentiles_on_sms_out_hourly','a')
    out.write(str(grid_idx)+',')
    for c in CC:
        out.write(str(np.sum(Calls_in<=c)/L_cc)+',')
    out.write('\n')
    out.close()
